In [2]:
import matplotlib.pyplot as plt
import numpy as np
import sqlite3
from matplotlib.ticker import EngFormatter

def xye(data):
    x = list(map(lambda x: x[0], data)) # n_summands
    y = list(map(lambda x: x[2] / 1e9, data)) # time_ns
    e = list(map(lambda x: x[3] / 1e9, data)) # stddev
    return x, y, e

db = 'file:project/benchmarks/results.db?mode=ro'
con = sqlite3.connect(db, uri=True)

def scatter_plot(run_id):
    settings = {
        "allreduce": {
            "fmt": "bo",
            "label": "MPI_Allreduce",
            "include": True,
        },
        "tree": {
            "fmt": "ro",
            "label": "Tree Reduce",
            "include": True
        },
        "reproblas": {
            "fmt": "co",
            "label": "ReproBLAS",
            "include": True
        }
    }
    f = plt.figure()
    ax = f.subplots(1)
    ax.set_ylabel('Accumulate Time')
    ax.set_xlabel('number of summands')

    formatter0 = EngFormatter(unit='s', places=2)
    formatter1 = EngFormatter(places=2)
    ax.yaxis.set_major_formatter(formatter0)
    ax.xaxis.set_major_formatter(formatter1)

    for mode, v in settings.items():
        if not v["include"]:
            continue

        fetch_data(mode)
        data = cur.fetchall()
        display(HTML(f"<h4>{mode}</h4>"))
        display_result(cur, data)

        ar_x, ar_y, ar_error = xye(data)
        ax.errorbar(ar_x, ar_y, yerr=ar_error, fmt=v["fmt"], label=v["label"], capsize=4.0)

        ticks = []
        for x in ar_x:
            lastX = ticks[-1] if len(ticks) > 0 else -1e10
            if x - lastX > 2e6:
                ticks.append(x)
        ax.xaxis.set_ticks(ticks)

    ax.legend(loc='upper left')

    ax.set_xlim(left=0)
    #ax.set_ylim(bottom=0)

    plt.show()

scatter_plot(69)

OperationalError: unable to open database file